In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# 設定 Selenium 瀏覽器參數
options = Options()
# 使用新版 headless 模式
options.add_argument("--headless=new")
# 避免 GPU 錯誤
options.add_argument("--disable-gpu")
# 避免 Linux 權限問題
options.add_argument("--no-sandbox")
# 防止 Selenium 被檢測
options.add_argument("--disable-blink-features=AutomationControlled")

# 自動安裝 ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 指定最多查詢筆數
MAX_RECORDS = 10
# 設定查詢商品
PRODUCT = "HFZ-B14A1FV"
_URL = f"https://www.momoshop.com.tw/search/searchShop.jsp?keyword={PRODUCT}"

print(f"🔍 正在搜尋商品：{PRODUCT}")
driver.get(_URL)

# 使用 WebDriverWait 等待商品列表加載
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "listArea"))
    )
    print("商品列表加載成功。")
except Exception as e:
    print(f"❌ 商品列表加載失敗: {e}")
    driver.quit()
    exit()

# 確認商品數量
product_elements = driver.find_elements(
    By.XPATH, "//ul[@class='listAreaUl']/li[@class='listAreaLi']"
)
num_products = len(product_elements)

if num_products == 0:
    print("⚠️ 沒有找到商品，結束程式。")
    driver.quit()
    exit()
elif num_products == 1:
    print("⚠️ 只有 1 個商品，將只抓取這筆資料。")

# 滾動頁面確保所有商品載入
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# 等待 JavaScript 完全載入
time.sleep(3)

# 爬取所有商品，最多抓取 `MAX_RECORDS` 個商品
for i in range(1, min(num_products, MAX_RECORDS) + 1):
    try:
        base_xpath = f"//ul[@class='listAreaUl']/li[@class='listAreaLi'][{i}]"

        # 產品資訊 XPath
        img_xpath = base_xpath + "//picture/img[@class='prdImg']"
        title_xpath = base_xpath + "//h3[@class='prdName']"
        price_xpath = base_xpath + "//span[@class='price']/b"

        # 取得商品圖片
        product_img = (
            WebDriverWait(driver, 5)
            .until(EC.presence_of_element_located((By.XPATH, img_xpath)))
            .get_attribute("src")
        )

        # 取得商品標題
        product_title = (
            WebDriverWait(driver, 5)
            .until(EC.presence_of_element_located((By.XPATH, title_xpath)))
            .text
        )

        # 取得商品價格
        product_price = (
            WebDriverWait(driver, 5)
            .until(EC.presence_of_element_located((By.XPATH, price_xpath)))
            .text
        )

        # 顯示結果
        print(f"📌 商品 {i}")
        print("📷 圖片：", product_img)
        print("🏷 標題：", product_title)
        print("💰 價格：", product_price)
        # 分隔
        print("-" * 50)

    except Exception as e:
        print(f"⚠️ 商品 {i} 無法取得：{e}")
        continue

# 關閉瀏覽器
driver.quit()
print("瀏覽器已關閉")

🔍 正在搜尋商品：HFZ-B14A1FV
商品列表加載成功。
📌 商品 1
📷 圖片： https://i4.momoshop.com.tw/1692293051/goodsimg/0010/326/654/10326654_OR_m.webp
🏷 標題： 【HERAN 禾聯】142L變頻直立式冷凍櫃(HFZ-B14A1FV)
💰 價格： 13,400
--------------------------------------------------
📌 商品 2
📷 圖片： https://i2.momoshop.com.tw/1721951162/goodsimg/0013/063/083/13063083_OR_m.webp
🏷 標題： 【HERAN 禾聯】142L 變頻直立式冷凍櫃(HFZ-B14A1FV)
💰 價格： 13,400
--------------------------------------------------
📌 商品 3
📷 圖片： https://i4.momoshop.com.tw/1727759234/goodsimg/0013/274/875/13274875_OR_m.webp
🏷 標題： 【HERAN 禾聯】142L 變頻直立式冷凍櫃(HFZ-B14A1FV)
💰 價格： 13,400
--------------------------------------------------
瀏覽器已關閉
